# REMEMBER to start the spark cluster with the little star icon before executing the notebook


# Hands-on Day 1

- course [slides](https://github.com/leggerf/MLCourse-1920/blob/master/Slides/Day1/ML-day1.pdf)

# You'll learn

- familiarize with jupyter notebooks, numpy and pandas

## Input data
- efficient data format: convert CSV to Parquet
- create input vector with features for MLLib. Format of the input depends on chosen ML library

## Visualization
- explore dataset, plot features
- correlation matrix


# Dataset description

The dataset used in this example is described [here](https://archive.ics.uci.edu/ml/datasets/HIGGS). It is a binary classification problem where the goal is to train a classifier able to distinguish between a signal process, the production of new theoretical Higgs bosons, and a background process with identical decay products but distinct kinematic features.

Each row of this dataset contains 28 features plus the label:

- 21 low-level features which represent the basic measure made by the particle detector
 -        Momentum of the observed paricles
 -        Missing transverse momentum
 -        Jets and b-tagging information
- 7 high-level features computed from the low-level features that encode the knowledge of the different intermediate states of the two processes (reconstructed invariant masses)

## Prepare the execution environment

Your code will run on a single dedicated server with 24 cores (hyperthreading enabled) and 192 GB of RAM. 
All the services needed for this tutorial are deployed as Kubernetes applications on this server. These include:
* JupytherHub
* Jupyter single-user servers
* the HDFS file-system
* Spark Clusters on demand 

In [1]:
!pip install pandas
!pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Check out these custom functions
from custom_functions import *

## Read in local csv files


In [3]:
inputFile = 'Higgs100k.csv'

#into panda DF
%time dataset = pd.read_csv(inputFile)

CPU times: user 852 ms, sys: 124 ms, total: 976 ms
Wall time: 988 ms


In [4]:
#then into spark dataframe...remember to start spark with the little icon

%time df = spark.createDataFrame(dataset)

CPU times: user 15.3 s, sys: 288 ms, total: 15.6 s
Wall time: 21.7 s


In [5]:
#In how many partitions is the dataframe distributed?
print("Number of partitions: {}".format(df.rdd.getNumPartitions()))
print("Partitioner: {}".format(df.rdd.partitioner))

Number of partitions: 2
Partitioner: None


# Now let's have a look at the input data

In [ ]:
%time total_events = df.count()

print('There are '+str(total_events)+' events')

In [ ]:
df.printSchema()

# Exercise 3

- Create a function that makes a plot of any of the above variable for signal versus background (using the label variable to discriminate)
  - see an example of the plot in the hands-on [slides](https://github.com/leggerf/MLCourse-1920/blob/master/Slides/Day1/ML-handson-day1.pdf)
  - the function should take as input the dataframe *df* and the variable name. For example `plotSignalvsBg(df, 'm_bb')`
  - to develop the code, use the 100k dataset, so that debugging goes quicker
- try to plot a few input variables and try to understand which ones are more promising to distinguish signal from background  

In [ ]:
#plot invariant mass distribution 
plotSignalvsBg(df, 'm_bb')

# Exercise 4 - Bonus

#### Create the input feature vector

- Libraries for ML tipically take as inputs data in a very specific format. Documentation on how to do data preprocessing in Spark: https://spark.apache.org/docs/latest/ml-features.html
- Try to add to the dataframe df a new column, called 'features' which is a vector column with all the variables above except for 'label'
   - features = [lepton_pT, lepton_eta, lepton_phi, ...]
   - Hint: look at the VectorAssembler transformer

In [ ]:
#create input vector for ML (features)
feature = df.columns
feature.remove('label')

#VectorAssembler is a transformer that combines a given list of columns into a single vector column
assembler = VectorAssembler(inputCols=feature, outputCol='features')
df = assembler.transform(df)

#take a look at how there is now a features column
display(df)
df.select('features').take(1)

#note also that features are already standardised between -1 and 1, so rescaling won't be needed

Print (or draw) the correlation matrix (a table showing correlation coefficients between variables)

In [ ]:
plotCorrelation(df, feature)

In [ ]:
#when you're done, stop spark, this will release the resources you're using
spark.stop()